## Question 1: **Answer**

In [1]:
import pandas as pd
import sqlite3

In [ ]:
def add_data_to_database(credit_rating, sales_people, customers, encounters):
    try:
        # Loading CSV files
        credit_rating = pd.read_csv(credit_rating)
        sales_people = pd.read_csv(sales_people)
        customers = pd.read_csv(customers)
        encounters = pd.read_csv(encounters)

        # Connect to the database and add data
        connection = sqlite3.connect("company_sales_records.db")
        credit_rating.to_sql("credit_rating", con=connection, index=False)
        sales_people.to_sql("sales_people", con=connection, index=False)
        customers.to_sql("customers", con=connection, index=False)
        encounters.to_sql("encounters", con=connection, index=False)
        connection.close()
        print("Data successfully added to the database.")
    except Exception as e:
        print(f"Error: {e}")

# Loading files
credit_ratings = "/content/sales_credit_rating.csv"
sales_people = "/content/sales_people.csv"
sales_customers = "/content/sales_customers.csv"
sales_encounters = "/content/sales_encounters.csv"

add_data_to_database(credit_ratings, sales_people, sales_customers, sales_encounters)

Data successfully added to the database.


In [3]:
%load_ext sql

In [4]:
# Connecting to the database
%sql sqlite:///company_sales_records.db

In [5]:
# Default to duplicated style
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

In [6]:
%%sql
SELECT * FROM credit_rating;

 * sqlite:///company_sales_records.db
Done.


CreditID,CreditDescription,MinFICO,MaxFICO,Comments
1,Extremely Poor,300.0,499.0,Cannot extend credit
2,Very Poor,500.0,580.0,Owner approval required to extend credit
3,Poor,580.0,619.0,Credit extended at extremely high interest rates
4,Fair,620.0,679.0,Credit extended at high interest rates
5,Good,680.0,699.0,Credit extended at normal interest rates
6,Very Good,700.0,850.0,Credit extended at low interest rates
7,Unknown,None,None,Paid cash without looking into financing options


In [7]:
%%sql
SELECT * FROM sales_people
LIMIT 5;

 * sqlite:///company_sales_records.db
Done.


SalesID,SFirstName,SLastName,SHireDate,Salary
1,Harry,Potter,13 02 1989,"$140,000.00"
2,Hermione,Granger,2 05 1989,"$82,000.00"
3,Ron,Weasley,2 05 1989,"$93,000.00"
4,Dudley,Dursley,27 04 1996,"$56,000.00"
5,Katie,Bell,15 05 1996,"$67,000.00"


In [8]:
%%sql
SELECT * FROM customers
LIMIT 5;

 * sqlite:///company_sales_records.db
Done.


CustomerID,CFirstName,CLastName,CPhone,AnnualIncome,CreditID
1,Clark,Adams,(801) 768-6043,"$64,250.00",3
2,Jessica,Johnson,(801) 763-9955,"$35,500.00",2
3,Owen,Neff,(801) 373-8051,"$33,750.00",2
4,Jeff,Raye,(801) 374-9170,"$30,000.00",2
5,Danny,Smith,(801) 766-9697,"$52,500.00",3


In [9]:
%%sql
SELECT * FROM encounters
LIMIT 5;

 * sqlite:///company_sales_records.db
Done.


EncID,SalesID,CustomerID,EncDate,Purchase
1,7,1,01-Jul-15,0
2,5,2,01-Jul-15,0
3,3,3,01-Jul-15,1
4,3,4,01-Jul-15,0
5,2,5,01-Jul-15,1


## Question 2: **Answer**

In [10]:
%%sql
UPDATE customers
SET AnnualIncome = REPLACE(REPLACE(AnnualIncome, '$', ''), ',', '');

 * sqlite:///company_sales_records.db
177 rows affected.


[]

In [11]:
%%sql
-- Main query to use the results of the two subqueries
SELECT
    rating_1_average_income,
    rating_6_average_income,
    -- Calculating percentage difference
    ROUND(rating_1_average_income/rating_6_average_income * 100, 2)
    AS percentage_diffference
FROM
    -- Subquery to calculate the average income for credit 1 rating
    (SELECT
    ROUND(AVG(AnnualIncome),2) AS rating_1_average_income
    FROM customers
    WHERE customers.CreditID = 1
    GROUP BY customers.CreditID)
    AS average_rating_1_income,

    -- Subquery to calculate the average income for credit 6 rating
    (SELECT
    ROUND(AVG(AnnualIncome),2) AS rating_6_average_income
    FROM customers
    WHERE customers.CreditID = 6
    GROUP BY customers.CreditID) AS average_rating_6_income;

 * sqlite:///company_sales_records.db
Done.


rating_1_average_income,rating_6_average_income,percentage_diffference
29500.0,59458.33,49.61


In [12]:
%%sql
SELECT
-- Calculating averages of credit rating 1 and 6
    ROUND(AVG(CASE WHEN customers.CreditID = 1 THEN AnnualIncome ELSE NULL END), 2)
    AS rating_1_average_income,
    ROUND(AVG(CASE WHEN customers.CreditID = 6 THEN AnnualIncome ELSE NULL END), 2)
    AS rating_6_average_income,

-- calculating the percentage credit rating 1 relative to rating 6
    ROUND((
        AVG(CASE WHEN customers.CreditID = 1 THEN AnnualIncome ELSE NULL END) /
        AVG(CASE WHEN customers.CreditID = 6 THEN AnnualIncome ELSE NULL END)
    ) * 100, 2) AS percentage_of_rating_6

FROM customers;

 * sqlite:///company_sales_records.db
Done.


rating_1_average_income,rating_6_average_income,percentage_of_rating_6
29500.0,59458.33,49.61


## Question 3: **Answer**

In [13]:
%%sql
SELECT
    customers.CreditID,
    customers.AnnualIncome,
    credit_rating.Comments
FROM customers
LEFT JOIN credit_rating
-- joining two tables on CreditID columns
ON customers.CreditID = credit_rating.CreditID
WHERE CFirstName = 'Sarah' AND CLastName = 'Scarborough';

 * sqlite:///company_sales_records.db
Done.


CreditID,AnnualIncome,Comments
1,28500.00,Cannot extend credit


## Question 4: **Answer**

In [14]:
%%sql
-- Counting sales per sales person
WITH sales_count AS (
    SELECT
        sales_people.SFirstName AS first_name,
        sales_people.SLastName AS last_name,
        sales_people.Salary AS salary,
        SUM(encounters.Purchase) AS total_sales
    FROM sales_people
    LEFT JOIN encounters
    ON sales_people.SalesID = encounters.SalesID
    GROUP BY sales_people.SalesID
)
-- Getting the person with the max and min sales
SELECT
    first_name,
    last_name,
    salary,
    printf('%20s', total_sales) AS sales_count
FROM sales_count
WHERE total_sales = (SELECT MAX(total_sales) FROM sales_count)
   OR total_sales = (SELECT MIN(total_sales) FROM sales_count);

 * sqlite:///company_sales_records.db
Done.


first_name,last_name,salary,sales_count
Hermione,Granger,"$82,000.00",14
Colin,Creevey,"$62,000.00",0


## Question 5: **Answer**

In [15]:
%%sql
SELECT
    customers.CFirstName As customer_first_name,
    customers.CLastName AS customer_last_name,
    customers.CreditID,
    sales_people.SFirstName AS salesperson_first_name,
    sales_people.SLastName As salesperson_last_name,
    encounters.EncDate AS encounter_date
FROM encounters
INNER JOIN customers
-- Joining encounters with customers
ON encounters.CustomerID = customers.CustomerID
INNER JOIN sales_people
--Joining encounters with sales_people
ON encounters.SalesID = sales_people.SalesID
    WHERE encounters.SalesID = 4 AND encounters.CustomerID = 8;

 * sqlite:///company_sales_records.db
Done.


customer_first_name,customer_last_name,CreditID,salesperson_first_name,salesperson_last_name,encounter_date
George,Campbell,5,Dudley,Dursley,02-Jul-15


## Question 6: **Answer**

In [16]:
%%sql
-- Joining two tables and aggregating cash purchases
WITH customer_cash_purchases AS (
    SELECT
        c.CFirstName AS first_name,
        c.CLastName AS last_name,
        c.CPhone AS phone_number,
        MAX(e.EncDate) AS latest_purchase_date,
        COUNT(e.EncDate) AS total_purchases
    FROM customers AS c
    INNER JOIN encounters AS e
        ON c.CustomerID = e.CustomerID
    WHERE c.CreditID  = 7
    GROUP BY c.CustomerID
)
-- Retrieving and labeling data
SELECT
    first_name,
    last_name,
    phone_number,
    latest_purchase_date,
    total_purchases,
    CASE
        WHEN total_purchases >=2  THEN 'High Priority'
        ELSE 'Standard Priority'
    END AS priority_label
FROM customer_cash_purchases;

 * sqlite:///company_sales_records.db
Done.


first_name,last_name,phone_number,latest_purchase_date,total_purchases,priority_label
Jeff,Wood,(801) 374-1669,03-Jul-15,1,Standard Priority
Colleen,Jacobson,(801) 785-4734,18-Jul-15,2,High Priority
Filipe,Diaz,(801) 374-2760,20-Jul-15,1,Standard Priority
Sam,Ogden,(801) 356-1120,28-Jul-15,2,High Priority
Ethan,Woodland,(801) 763-1634,26-Jul-15,1,Standard Priority
Alison,Jameson,(801) 356-6198,30-Jul-15,1,Standard Priority
